## yellow_tripdata_2016-01.csv
### 1.6 G

In [46]:
sc

In [47]:
import pyproj
import csv
import shapely.geometry as geom
import fiona
import fiona.crs
import shapely
import rtree
import geopandas as gpd
import numpy as np
import operator
# just for display, not for calculation
import pandas as pd

In [2]:
df = pd.read_csv('./df_shuffle.csv')
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2016-01-19 09:57:48,2016-01-19 10:15:50,3,1.16,-73.984688,40.759109,1,N,-74.000557,40.757912,1,12.0,0.0,0.5,2.56,0.0,0.3,15.36
1,2,2016-01-15 04:00:16,2016-01-15 04:02:03,1,0.76,-73.988274,40.731468,1,N,-73.981339,40.741234,2,4.5,0.5,0.5,0.00,0.0,0.3,5.80
2,2,2016-01-10 00:58:03,2016-01-10 01:04:49,1,1.28,0.000000,0.000000,1,N,0.000000,0.000000,2,6.5,0.5,0.5,0.00,0.0,0.3,7.80
3,1,2016-01-03 17:32:54,2016-01-03 18:12:50,1,5.80,-74.017334,40.704903,1,N,-73.990944,40.663696,1,28.5,1.0,0.5,6.05,0.0,0.3,36.35
4,1,2016-01-06 08:21:41,2016-01-06 08:42:30,1,2.70,-74.001259,40.741528,1,N,-73.971085,40.760841,2,15.0,0.0,0.5,0.00,0.0,0.3,15.80


In [29]:
import datetime
def wkd(time):
    return datetime.datetime.strptime(time, '%Y-%m-%d %H:%M:%S').weekday()

In [40]:
# wkd(df.tpep_pickup_datetime[0])
# pd.to_datetime(df.tpep_pickup_datetime).map(lambda x: x.weekday())

# Function

In [58]:
def countLine(partID, records):
    import pyproj
    import csv
    import shapely.geometry as geom
    import fiona
    import fiona.crs
    import shapely
    import rtree
    import geopandas as gpd
    import numpy as np
    import operator
    import pandas as pd
    import datetime
    
    shapefile = '../why_yellow_taxi/Buffer/entr_buffer_100_feet_epsg4269_nad83/entr_buffer_100_feet_epsg4269_nad83.shp'
    entr_buf = gpd.read_file(shapefile)
    entr_buf = entr_buf.to_crs(fiona.crs.from_epsg(2263))
    
    routes = ['Route_' + str(n) for n in range(1, 12)]
    entr2line = []
    for i in xrange(len(entr_buf)):
        lines = []
        for line in list(entr_buf.loc[:,routes].ix[i].dropna().values):
            try:
                line = str(int(line))
            except ValueError:
                pass
            lines.append(line)
        entr2line.append(lines)
    entr_buf['entr2line'] = entr2line
    
    index = rtree.Rtree()
    for idx, geometry in enumerate(entr_buf.geometry):
        index.insert(idx, geometry.bounds)
    

    entr_pair = {}
    pick_entr = {}
    drop_entr = {}
    entr_lines = {}
    
    proj = pyproj.Proj(init='epsg:2263', preserve_units=True)
    
    if partID==0:
        records.next()
    reader = csv.reader(records)
    for row in reader:
        if ((float(row[5])!=0) and float(row[9]!=0)):
            if row[1]:
                wd = datetime.datetime.strptime(row[1], '%Y-%m-%d %H:%M:%S').weekday()
    
            p = geom.Point(proj(float(row[5]), float(row[6])))
            d = geom.Point(proj(float(row[9]), float(row[10])))
            p_potential = index.intersection((p.x,p.y,p.x,p.y))
            d_potential = index.intersection((d.x,d.y,d.x,d.y))
            p_match = None # The first one match, should be the closest one? No!
            d_match = None
            
            for p_idx in p_potential:
                if entr_buf.geometry[p_idx].contains(p):
                    p_match = p_idx # print 'p',p_idx
                    p_lines = set(entr_buf.entr2line[p_idx])
                    break
            pick_entr[p_match] = pick_entr.get(p_match, 0)+1
            
            for d_idx in d_potential:
                if entr_buf.geometry[d_idx].contains(d):
                    d_match = d_idx # print 'd',d_idx
                    d_lines = set(entr_buf.entr2line[d_idx])
                    break
            drop_entr[d_match] = drop_entr.get(d_match, 0)+1
            
            if ((p_match and d_match) and (p_match != d_match)):
                dirct_lines = tuple(p_lines.intersection(d_lines))
                #dirct_lines_wd = zip(dirct_lines, wd)
                if dirct_lines:
                    entr_lines[dirct_lines] = entr_lines.get(dirct_lines, 0)+1
                    #entr_lines[dirct_lines_wd] = entr_lines.get(dirct_lines_wd, 0)+1
                if p_match > d_match:
                    pair = (d_match, p_match)
                else:
                    pair = (p_match, d_match)
                entr_pair[pair] = entr_pair.get(pair, 0)+1
                
    return entr_lines.items()
    
    
        

In [61]:
def mapper(record):
    for key in record[0]:
        yield key, record[1]
        
rdd = sc.textFile('./df_shuffle.csv')
counts = rdd.mapPartitionsWithIndex(countLine)#.flatMap(mapper).reduceByKey(lambda x,y: x+y).collect()  

In [63]:
counts.collect()

[((u'Q', u'R', u'N'), 1),
 (('6',), 2),
 ((u'C',), 1),
 ((u'F',), 4),
 (('1',), 6),
 (('1', '3', '2'), 1),
 ((u'L',), 2),
 ((u'A', u'C', u'E'), 1),
 ((u'E', u'M'), 1),
 ((u'Q', u'R', u'N'), 3),
 (('6',), 4),
 ((u'N', u'Q', u'R', '5', '4', '6'), 1),
 ((u'M',), 1),
 ((u'C', u'E'), 2),
 (('1',), 5),
 ((u'A', '1', u'C'), 1),
 ((u'C',), 1),
 (('1', '3', '2'), 4),
 ((u'L',), 1),
 ((u'A', '1', u'C', u'B', u'D'), 1),
 ((u'A', u'C', '2', '3'), 1),
 ((u'A', u'C', u'E'), 3),
 (('7',), 2)]

In [14]:
def mapper(record):
    for key in record[0]:
        yield key, record[1]

#rdd = sc.textFile('./yellow_tripdata_2016-01.csv')
counts = rdd.mapPartitionsWithIndex(countLine).flatMap(mapper).reduceByKey(lambda x,y: x+y).collect()  

In [17]:
sorted(counts, key=lambda x: x[1], reverse=True)

[('1', 11795),
 (u'C', 8512),
 (u'E', 6547),
 ('6', 6464),
 (u'R', 6334),
 (u'F', 5823),
 (u'N', 5601),
 (u'M', 5282),
 (u'B', 4717),
 ('2', 4557),
 ('3', 4547),
 (u'A', 4421),
 (u'Q', 4300),
 (u'D', 3290),
 ('7', 2472),
 (u'L', 1507),
 ('5', 1270),
 ('4', 1256),
 (u'J', 422),
 (u'Z', 351),
 (u'GS', 191),
 (u'S', 182),
 (u'G', 105)]

![](http://web.mta.info/maps/images/subway_map_2400x2946opt.jpg)